# Eflux2 applied to *A. niger*

In [1]:
import sys
import pandas as pd
import cobra
sys.path.append('../../src')
from eflux2 import EFlux2

## Read global proteomics normalized to the reference strain SF ABF180_17_R2

[(βAl-3HP)++, pyc2++, Δald6a+NeoR] (Strain ABF-10216R8, ICE ID ABF_011245)

In [2]:
global_prot = pd.read_csv('../../data/round2/normalized_global_proteomics.csv', index_col='Transcript')
global_prot

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
Transcript,,,,,,,,,,,,,,,,,,,,,
1170085,0.967266,0.996633,0.992893,0.988900,1.005546,0.994246,0.995436,0.984235,0.985065,0.987202,...,0.996191,1.000241,0.981752,1.003174,1.000581,1.006389,0.998983,0.974427,0.986005,0.982189
1141495,1.012991,1.010041,1.011290,1.012467,1.008045,1.010177,1.032464,1.009299,1.012114,1.014553,...,1.010923,1.019918,1.003567,1.020978,1.010532,1.022859,1.026123,1.015454,1.016219,1.003530
201546,0.979618,0.980405,0.973220,0.981140,0.998126,0.998079,0.991908,0.981119,0.990187,0.990484,...,0.988765,0.994606,0.979760,0.993260,1.001902,0.996684,0.994010,0.982122,0.986781,0.980817
1147898,0.995988,1.000736,0.999673,1.021090,1.020466,1.023879,1.007351,1.005843,1.003600,1.006999,...,1.007180,1.001447,1.008006,1.004574,1.003261,1.014986,1.022990,1.002505,0.996382,1.002931
1142869,1.018378,1.017542,1.017273,1.015201,1.009343,1.003975,1.016892,1.010228,1.026211,1.010030,...,1.000423,1.002124,1.005939,1.004112,1.001554,1.010091,0.992440,1.000478,0.998578,0.997856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117615,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
210213,1.000000,1.000000,1.000000,1.000000,1.000000,0.926492,0.928163,1.000000,0.951025,1.000000,...,1.013350,1.024441,1.026118,1.015304,1.012428,0.997734,0.990888,0.970595,1.000000,0.954842
1119048,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Read iJB1325 with non-native genes added for 3HP production

In [3]:
# todo: update gurobi license (not required to use gurobi)
model = cobra.io.load_json_model('../../models/iJB1325_HP.nonnative_genes.pubchem.flipped.nonzero.reduced.json')
#model.solver = 'gurobi'
#model.solver

In [4]:
model.summary()

Non-linear or non-reaction model objective. Falling back to minimal display.


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


### No objective function is set for FBA
The biomass reaction is missing in the model. Use CO2 evolution as the objective function since CO2 is the only major organic product not measured

In [5]:
with model:
    model.objective = 'BOUNDARY_CO2e'
    model.reactions.BOUNDARY_GLCe.upper_bound = 10.0
    model.reactions.EX_3hpp_e.lower_bound = 10.0
    display(model.summary())

Metabolite,Reaction,Flux,C-Number,C-Flux
GLCe,BOUNDARY_GLCe,10,6,100.00%
O2e,BOUNDARY_O2e,21.19,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CO2e,BOUNDARY_CO2e,-30,1,50.00%
H2Oe,BOUNDARY_H2Oe,-12.39,0,0.00%
He,BOUNDARY_He,-17.61,0,0.00%
3hpp_e,EX_3hpp_e,-10,3,50.00%


In [6]:
model.objective = 'BOUNDARY_CO2e'

## Read line rates

In [7]:
#line_rates = pd.read_csv('../../data/round2/Line_rates_include_0hr.csv', index_col='Line Name')
line_rates = pd.read_csv('../../data/round2/Line_rates_exclude_0hr.csv', index_col='Line Name')
line_rates.head()

,Omics Sample ID,Strain (ICE),Genotype,growth_rate,glucose_uptake_rates (mmol/gDCW * hr),3hpacid_secretion_rates (mmol/gDCW * hr),ethanol_uptake_rates (mmol/gDCW * hr),erythritol_secretion_rates (mmol/gDCW * hr),citricacid_uptake_rates (mmol/gDCW * hr)
Line Name,,,,,,,,,
SF ABF180_1_R1,SF ABF180_73,ABF_008340,wild-type,NaN,3.824934,0.000000,0.792616,0.003407,0.013740
SF ABF180_1_R2,SF ABF180_74,ABF_008340,wild-type,NaN,3.557246,0.000000,0.678012,0.012109,0.009979
SF ABF180_1_R3,SF ABF180_75,ABF_008340,wild-type,NaN,3.415469,0.000000,1.080535,0.010234,0.009554
SF ABF180_2_R1,SF ABF180_76,ABF_008343,(βAl-3HP)+,NaN,5.928539,2.040350,0.352739,0.022628,-0.001312
SF ABF180_2_R2,SF ABF180_77,ABF_008343,(βAl-3HP)+,NaN,5.792073,2.099251,0.539265,0.005308,-0.002935


In [8]:
# TODO: update ref line and strain for round 2
ref_line = 'SF ABF180_17_R2'

ref_line_glucose_rate = line_rates.loc[ref_line,'glucose_uptake_rates (mmol/gDCW * hr)']
ref_line_glucose_rate

6.277123454

## Normalize uptake and secretion rates by glucose uptake rate of reference study line.

In [9]:
normalized_line_rates = line_rates.drop(['Omics Sample ID', 'Strain (ICE)', 'Genotype'], axis=1)
normalized_line_rates = normalized_line_rates.divide(normalized_line_rates.loc[ref_line, 'glucose_uptake_rates (mmol/gDCW * hr)'], axis=1)
normalized_line_rates.to_csv('../../data/round2/normalized_line_rates.csv')
normalized_line_rates

,growth_rate,glucose_uptake_rates (mmol/gDCW * hr),3hpacid_secretion_rates (mmol/gDCW * hr),ethanol_uptake_rates (mmol/gDCW * hr),erythritol_secretion_rates (mmol/gDCW * hr),citricacid_uptake_rates (mmol/gDCW * hr)
Line Name,,,,,,
SF ABF180_1_R1,NaN,0.609345,0.000000,0.126271,0.000543,0.002189
SF ABF180_1_R2,NaN,0.566700,0.000000,0.108013,0.001929,0.001590
SF ABF180_1_R3,NaN,0.544114,0.000000,0.172139,0.001630,0.001522
SF ABF180_2_R1,NaN,0.944467,0.325045,0.056194,0.003605,-0.000209
SF ABF180_2_R2,NaN,0.922727,0.334429,0.085910,0.000846,-0.000467
...,...,...,...,...,...,...
SF ABF180_23_R2,NaN,0.989133,0.478897,0.121175,0.012959,0.001360
SF ABF180_23_R3,NaN,1.137562,0.564211,0.150258,0.013712,0.001255
SF ABF180_24_R1,NaN,1.124896,0.949422,0.043847,0.024140,0.002688


### Need to add ethanol, erythritol, and citric acid secretion to the model

In [11]:
# TODO: update the reduced model and add constraints for ethanol, erythritol, and citric acid
# Use glucose uptake and 3HP secretion for now
fluxes = {}
for rep in global_prot.columns:
    with model:
        glucose_uptake = normalized_line_rates.loc[rep,'glucose_uptake_rates (mmol/gDCW * hr)' ]
        secrete_3hp = normalized_line_rates.loc[rep,'3hpacid_secretion_rates (mmol/gDCW * hr)']
        if glucose_uptake:
            model.reactions.BOUNDARY_GLCe.upper_bound = glucose_uptake
        if secrete_3hp:
            model.reactions.EX_3hpp_e.lower_bound =  secrete_3hp
        try:
            print(rep)
            fluxes[rep] = EFlux2(model, global_prot[rep])
        except TypeError:
            print(f"Replicate {rep} with glucose {glucose_uptake} and 3hp {secrete_3hp} is infeasible")

SF ABF180_1_R1
FBA status optimal
FBA solution 2.2060924843278213
EFlux2 status optimal
EFlux2 solution 43.16552443203364
SF ABF180_1_R2
FBA status optimal
FBA solution 2.136167562532686
EFlux2 status optimal
EFlux2 solution 39.99940392698337
SF ABF180_1_R3
FBA status optimal
FBA solution 2.0966236901100888
EFlux2 status optimal
EFlux2 solution 38.46954312160907
SF ABF180_2_R1
FBA status optimal
FBA solution 2.2917979691568258
EFlux2 status optimal
EFlux2 solution 54.682889319132585
SF ABF180_2_R2
FBA status optimal
FBA solution 2.2710127253287564
EFlux2 status optimal
EFlux2 solution 53.38622765179727
SF ABF180_2_R3
FBA status optimal
FBA solution 2.320797179984593
EFlux2 status optimal
EFlux2 solution 57.02180500940547
SF ABF180_3_R1
FBA status optimal
FBA solution 2.1112534229456594
EFlux2 status optimal
EFlux2 solution 49.20506558013514
SF ABF180_3_R2
FBA status optimal
FBA solution 2.1662868751801234
EFlux2 status optimal
EFlux2 solution 51.264034731877736
SF ABF180_3_R3
FBA statu

/Users/kimj972/anaconda3/envs/python3_cobrapy/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


FBA status infeasible
FBA solution 0.2806731337371398
EFlux2 status infeasible
EFlux2 solution 33.83739856414726
SF ABF180_5_R2
FBA status infeasible
FBA solution 0.07592685489864517
EFlux2 status infeasible
EFlux2 solution 33.763326604652846
SF ABF180_5_R3
FBA status infeasible
FBA solution 0.6505052096787254
EFlux2 status infeasible
EFlux2 solution 45.12344023979455
SF ABF180_6_R1
FBA status optimal
FBA solution 1.9736879237815423
EFlux2 status optimal
EFlux2 solution 40.953686990914136
SF ABF180_6_R2
FBA status optimal
FBA solution 2.116037224570294
EFlux2 status optimal
EFlux2 solution 47.97921762112371
SF ABF180_6_R3
FBA status optimal
FBA solution 2.1601795288034955
EFlux2 status optimal
EFlux2 solution 48.89344342144593
SF ABF180_7_R1
FBA status optimal
FBA solution 1.8562235913819487
EFlux2 status optimal
EFlux2 solution 41.37332856046809
SF ABF180_7_R2
FBA status optimal
FBA solution 1.7589641652929604
EFlux2 status optimal
EFlux2 solution 44.36561267035181
SF ABF180_7_R3
FBA 

In [12]:
fluxes

{'SF ABF180_1_R1': <Solution 43.166 at 0x7fe88318ebe0>,
 'SF ABF180_1_R2': <Solution 39.999 at 0x7fe883885940>,
 'SF ABF180_1_R3': <Solution 38.470 at 0x7fe882115a90>,
 'SF ABF180_2_R1': <Solution 54.683 at 0x7fe884451a58>,
 'SF ABF180_2_R2': <Solution 53.386 at 0x7fe8848f3828>,
 'SF ABF180_2_R3': <Solution 57.022 at 0x7fe884b74668>,
 'SF ABF180_3_R1': <Solution 49.205 at 0x7fe8854d9208>,
 'SF ABF180_3_R2': <Solution 51.264 at 0x7fe88595ddd8>,
 'SF ABF180_3_R3': <Solution 49.033 at 0x7fe88536cc18>,
 'SF ABF180_4_R1': <Solution 51.854 at 0x7fe886b11da0>,
 'SF ABF180_4_R2': <Solution 47.415 at 0x7fe8873eab38>,
 'SF ABF180_4_R3': <Solution 45.966 at 0x7fe884b74f60>,
 'SF ABF180_5_R1': <Solution infeasible at 0x7fe884eb5400>,
 'SF ABF180_5_R2': <Solution infeasible at 0x7fe8882ec160>,
 'SF ABF180_5_R3': <Solution infeasible at 0x7fe884e979b0>,
 'SF ABF180_6_R1': <Solution 40.954 at 0x7fe887e13710>,
 'SF ABF180_6_R2': <Solution 47.979 at 0x7fe88ab36940>,
 'SF ABF180_6_R3': <Solution 48.893 

In [ ]:
# TODO: update Eflux to handle infeasible cases

In [ ]:
# TODO: update for round 2
efluxes = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['fluxes']) for rep in fluxes]))
reduced_costs = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['reduced_costs']) for rep in fluxes]))
efluxes.to_csv('Eflux2_flux_yield.csv')
reduced_costs.to_csv('Eflux2_yield_reduced_costs.csv')
eflux_rates = efluxes*ref_line_glucose_rate
eflux_rates.to_csv('Learn/Eflux2_flux_rates.csv')

In [ ]:
eflux_rates